In [59]:
import re

f = open("viaf88890045.003.perseus-eng1.xml")
outf = open('bio-work1.xml','w')

entrylens = {}

inheader = 1
needspace = 0

curentryid = ''
for l in f:
    if(inheader):
        if(re.search('</teiHeader>',l)):
            inheader = 0
        print(l,end='',file=outf)
        continue
        
    m = re.search('entry" xml:id="([^"]+)"',l)
    if(m):
        curentryid = m[1]
        entrylens[curentryid] = 0
        
    if(curentryid in entrylens):
        entrylens[curentryid] = entrylens[curentryid] + len(re.sub('<[^<]+>','',l).split())


    l = re.sub('\s*\n',' ',l)
    l = re.sub('^\s+','',l)
    l = re.sub('([,;])</hi>','</hi>\g<1>',l)
    if(1 and re.search('^\s*(<div|<by|<p>|</div>)',l)):
        l = re.sub('^\s*(<div|<by|<p>|</div>)','\n\n\g<1>',l)
    if(re.search('^\s*(<head|<milestone)',l)):
        l = re.sub('^\s*(<head|<milestone)','\n\g<1>',l)
    print(l,end='',file=outf)

f.close()
outf.close()

singnameindex = {}
for foo in entrylens:
    workid = re.sub('-bio.*','',foo)
    for tmpkey in workid.split('-'):
        if(len(tmpkey)<3):
            continue
        if(not tmpkey in singnameindex):
            singnameindex[tmpkey] = {}
        singnameindex[tmpkey][foo] = entrylens[foo]

f = open('bio-work1.xml')
outf = open('bio-work.xml','w')

entryheads = {}
entrydates = {}
entrytitles = {}
sentences = 0
inheader = 1
curpage = ''
curvolume = ''
curentryid = ''

shortnames = ['Cato','Hera','Jew','Jews','Livy','Milo','Num','Otho','Ovid','Piso','Pius','Way','Ways','West','Zeno','Zeus']
shortpat = '('+ '|'.join(shortnames) + ')'

newdates = 0
newdates1 = 0
newdates2 = 0

centval = {}
centval["15th B.C."] = "-1500-1401"
centval["twelfth B.C."] = "-1200-1101"
centval["tenth B.C."] = "-1000-901"
centval["eleventh B.C."] = "-1100-1001"
centval["ninth B.C."] = "-900-801"
centval["8th B.C."] = "-800-701"
centval["eighth B.C."] = "-800-701"
centval["seventh B.C."] = "-700-601"
centval["7th B.C."] = "-700-601"
centval["sixth B.C."] = "-600-501"
centval["6th B.C."] = "-600-501"
centval["5th B.C."] = "-500-401"
centval["fifth B.C."] = "-500-401"
centval["fourth B.C."] = "-100-101"
centval["4th B.C."] = "-400-301"
centval["3rd B.C."] = "-300-201"
centval["third B.C."] = "-300-201"
centval["second B.C."] = "-200-101"
centval["1st B.C."] = "-100-001"
centval["first B.C."] = "-100-001"
centval["a B.C."] = "-100-001"
centval["the B.C."] = "-100-001"
centval["last B.C."] = "-100-001"
centval["first A.D."] = "001-100"
centval["2nd A.D."] = "101-200"
centval["second A.D."] = "101-200"
centval["3rd A.D."] = "201-300"
centval["third A.D."] = "201-300"
centval["4th A.D."] = "301-400"
centval["fourth A.D."] = "301-400"
centval["fifth A.D."] = "401-500"
centval["5th A.D."] = "401-500"
centval["sixth A.D."] = "501-600"
centval["6th A.D."] = "501-600"
centval["seventh A.D."] = "601-700"
centval["7th A.D."] = "601-700"
centval["eighth A.D."] = "701-800"
centval["8th A.D."] = "701-800"
centval["ninth A.D."] = "801-900"

centval["9th A.D."] = "801-900"
centval["10th A.D."] = "901-1000"
centval["tenth A.D."] = "901-1000"
centval["eleventh A.D."] = "1001-1100"
centval["12th A.D."] = "1101-1200"
centval["twelfth A.D."] = "1101-1200"
centval["thirteenth A.D."] = "1201-1300"
centval["13th A.D."] = "1201-1300"
centval["fourteenth A.D."] = "1301-1400"
centval["14th A.D."] = "1301-1400"
centval["fifteenth A.D."] = "1401-1500"
centval["15th A.D."] = "1401-1500"
centval["sixteenth A.D."] = "1501-1600"
centval["16th A.D."] = "1501-1600"

goodxrefs = 0
ambxrefs = 0
xrefs = 0
for l in f:
    l = re.sub('<hi rend="ital">([A-Z][a-z]+\. [^<]+)</hi>','<title>\g<1></title>',l)
    m = re.search('<milestone unit="volume" n="([0-9]+)"',l)
    l,hits = re.subn('(\s+|\()(A\.\s*D\.)\s+([0-9]+)','\g<1><date when-custom="\g<3>">\g<2> \g<3></date>',l)
    newdates1 = newdates1 + hits
    l,hits = re.subn('(\s+|\()(B\.\s*C\.)\s+([0-9]+)','\g<1><date when-custom="-\g<3>">\g<2> \g<3></date>',l)
    newdates1 = newdates1 + hits
    l,hits = re.subn('([0-9]+)\s+(B\.\s*C\.)','<date when-custom="-\g<1>">\g<1> \g<2></date>',l)
    newdates1 = newdates1 + hits
    if(m):
        curvolume = m[1]
        
    m = re.search('<pb n="([0-9]+)"',l)
    if(m):
        curpage = m[1]
        
    if(re.search('<head',l)):
        curhead = re.sub('<[^>]+>','',l)
        curhead = re.sub('\n','',curhead)
        if( not curentryid in entryheads):
            entryheads[curentryid] = curhead
        
    m = re.search('entry" xml:id="([^"]+)"',l)
    if(m):
        curentryid = m[1]
        #entrylens[curentryid] = 0
        entrydates[curentryid] = []
        entrytitles[curentryid] = []
        
    if(inheader):
        if(re.search('</teiHeader>',l)):
            inheader = 0
        print(l,end='',file=outf)
        continue

    i = 0 
    matchs = '<date when-custom="([0-9]+|\-[0-9]+)">([^<]+</date>)(\s+and\s+|\s+or\s+|\s+to\s+|,\s+)([0-9]+|\-[0-9]+)'
    m = re.search(matchs,l)
    while(re.search(matchs,l)):
        m = re.search('matchs,l)
        newdates = newdates + 1
        date1 = m[1]
        linkword = m[3]
        date2 = m[4]
        if(re.search('^\-',date1)):
            date2 = '-' + date2
            #print('date2 now',date1,date2,l)
        i = i + 1
        if(i> 100):
            print('outofcontrol1',l)
            break
        if(0 and int(date2) < int(date1)):
            print('newdate',curentryid,curvolume,curpage,date1,linkword,date2,l,end='')
        l = re.sub(matchs,l,1)


    l,hits = re.subn('(</date>)\s*-\s*([0-9]+)','\g<1>-<date when-custom="\g<2>">\g<2></date>',l)
    newdates2 = newdates2 + hits
    
    workl = l
    matchs = '<date when-custom="([0-9\-]+)">([^<]+)</date>-<date when-custom="([0-9\-]+)">'
    while(re.search(matchs,workl)):
        m = re.search(matchs,workl)
        date1 = m[1]
        date2 = m[3]
        if(not len(date1) == len(date2) and int(date1) > int(date2)):
            datemask = re.sub('(.)','.',date2) + '$'
            half1 = re.sub(datemask,'',date1)
            newdate2 = half1 + date2
            if(1 or int(newdate2) < int(date1)):
                print('unmatched',date1,date2,newdate2)
            l = re.sub('when-custom="' + date2+ '"','when-custom="' + newdate2 + '"',l)
        workl = re.sub(matchs,'',workl,1)


        
    
    workl = l
    while(re.search('<title[^>]*>([^<]+)',workl)):
        m = re.search('<title[^>]*>([^<]+)',workl)
        if(curentryid in entrytitles):
            entrytitles[curentryid].append(m[1])
        workl = re.sub('<title[^>]*>([^<]+)','',workl,1)
        
    workl = l
        
    l = re.sub('(<hi rend="smallcaps">[^<]+)</hi>\s+<hi rend="smallcaps">','\g<1> ',l)
    if(re.search('<p>',l)):
        l = re.sub('([A-Z][a-z][a-z]\.)','<abbr>\g<1></abbr>',l)
        l = re.sub('<abbr>'+shortpat+'\.</abbr>','\g<1>',l)
        while(re.search('(="[^"]*|<bibl[^>]*>[^<]*)<[/]*abbr>',l)):
            l = re.sub('(="[^"]*|<bibl[^>]*>[^<]*)<[/]*abbr>','\g<1>',l)
        l,hits = re.subn('([a-z][a-z]+[\.!?"])\s+([A-Z][a-z]*\s+[A-Za-z]+[,\s]|[A-Z][a-z]*\s+<[a-zA-Z]+)','\g<1>\n<s n="r01"/>\g<2>',l)
        l,hits = re.subn('(\)\.|\.\)|\.</hi>\)|\.</foreign>\))\s+([A-Z])','\g<1>\n<s n="r2"/>\g<2>',l)
        l,hits = re.subn('(</date>\.)\s+([A-Z])','\g<1>\n<s n="r3"/>\g<2>',l)
        l,hits = re.subn('(\]\.|\.\)])\s+([A-Z])','\g<1>\n<s n="r4"/>\g<2>',l)
        l,hits = re.subn('([a-z][a-z]+[\.!?"])\s+([A-Z][a-z]*,\s+[A-Za-z]+[,\s])','\g<1>\n<s n="r5"/>\g<2>',l)
        l,hits = re.subn('([a-z][a-z]+[\.!?"])\s+([A-Z][a-z]*\s+\()','\g<1>\n<s n="r6"/>\g<2>',l)
        l,hits = re.subn('(\.</hi>\])\s+([A-Z])','\g<1>\n<s n="r7"/>\g<2>',l)
        l,hits = re.subn('(\.)\s+(In\s+<date)','\g<1>\n<s n="r8"/>\g<2>',l)
        l,hits = re.subn('(\.</foreign>)\s+([A-Z])','\g<1>\n<s n="r9"/>\g<2>',l)
        l,hits = re.subn('(</ref>\]|</bibl>\))\s+([A-Z])','\g<1>\n<s n="r11"/>\g<2>',l)

    
    matchs = '([a-zA-Z0-9]+)\s+(century)\s+([AB]\.\s*[DC]\.|after\s+Christ|before Christ)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        if(m[3] == 'after Christ'):
            era = 'A.D.'
        elif(m[3] == 'before Christ'):
            era = 'B.C.'
        else:
            era = m[3]
        curkey = m[1] + ' ' + era
        curkey = re.sub('\.\s+','.',curkey)
        if(not curkey in centval):
            print('missingcentval',curkey,l)
            curval = 'noval'
        else:
            curval = centval[curkey]
        l = re.sub(matchs,'<date when-custom="'+curval+'">\g<1> \g<2> <z/>\g<3></date>',l,1)
    l = re.sub('<z/>','',l)
    
    olymppat = '([0-9]+)([a-z]+\s+Olympiad)'
    while(re.search(olymppat,l)):
        m = re.search(olymppat,l)
        ol = int(m[1])
        startyear = -776 + (4*(ol-1))
        endyear = startyear + 3
        yearrange = str(startyear) + '-' + str(endyear)
        l = re.sub(olymppat,'<date when-custom="'+yearrange+'">\g<1><z/>\g<2></date>',l,1)
    l = re.sub('<z/>','',l)
    
    olymppat2 = '(\(Ol\.\s+)([0-9]+)'
    while(re.search(olymppat2,l)):
        m = re.search(olymppat2,l)
        ol = int(m[2])
        startyear = -776 + (4*(ol-1))
        endyear = startyear + 3
        yearrange = str(startyear) + '-' + str(endyear)
        l = re.sub(olymppat2,'(<date when-custom="'+yearrange+'">\g<1><z/>\g<2></date>',l,1)
    l = re.sub('<z/>','',l)
 
    i = 0
    workl = l
    while(re.search('<date when-custom="([\-0-9]+)"',workl)):
        m = re.search('<date when-custom="([\-0-9]+)"',workl)
        i = i + 1
        if(i> 100):
            print('outofcontrol12',l)
            break
        entrydates[curentryid].append(m[1])
        workl = re.sub('<date when-custom="([\-0-9]+)"','',workl,1)

    if( re.search('<p>\[<hi rend="smallcaps">[^<]+</hi>\.\]</p>',l)):
            print(l,end='',file=outf)
            continue
            

    matchs = '(<hi rend="smallcaps">)([^<]+)(</hi>)(,\s+No\.\s+)([0-9]+)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        curname = m[2]
        curnum = m[5]
        xrefkey = re.sub('[,\s]+','-',curname.lower()) + '-bio-' + curnum
        xrefs = xrefs + 1
        if(xrefkey in entrylens):
            goodxrefs = goodxrefs + 1
            res = 'hit'
            l = re.sub(matchs,'<ref target="'+xrefkey+'">\g<2></ref>\g<4>\g<5>',l,1)
        else:
            res = 'fail'
            print('xref',res,curentryid,curname,curnum,xrefkey,xrefs,goodxrefs)
            l = re.sub(matchs,'<hix rend="smallcaps">\g<2></ref>\g<4>\g<5>',l,1)
            
    matchs = '(<hi rend="smallcaps">)([^<]+)(</hi>)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        curname = m[2]
        xrefkey1 = re.sub('[,\.\s]+','-',curname.lower()) + '-bio-1'
        xrefkey2 = re.sub('[,\.\s]+','-',curname.lower()) + '-bio-2'
        xrefkey1 = re.sub('[\-]+','-',xrefkey1)
        xrefkey2 = re.sub('[\-]+','-',xrefkey2)
        xrefs = xrefs + 1
        if(xrefkey1 in entrylens):
            if(xrefkey2 in entrylens):
                res ='amb'
                ambxrefs = ambxrefs + 1

            else:
                res = 'hit'
                goodxrefs = goodxrefs + 1
                
            l = re.sub(matchs,'<ref target="'+xrefkey1+'">\g<2></ref>',l,1)
        else:
            res = 'fail'
            l = re.sub(matchs,'<hix rend="smallcaps">\g<2></hi>',l,1)
        if(not res == 'hit'):
            print('xref2',res,curentryid,curname,xrefkey1,xrefs,goodxrefs,ambxrefs)


    l = re.sub('<hix ','<hi ',l)
    print(l,end='',file=outf)
outf.close()

print('done')


SyntaxError: EOL while scanning string literal (<ipython-input-59-3c1529570a2c>, line 167)

In [12]:
print('newdates1',newdates1,'newdates',newdates,'newdates2',newdates2,'sents',sentences)

i = 0
for foo in sorted(entrylens,key=entrylens.get,reverse=True):
    #if(1 and len(entrydates[foo])>0):
   #     continue
    i = i + 1
    if(i>1000):
        continue
    #if(entrylens[foo] < 1000):
    #    continue
    if(len(entrydates[foo]) == 0):
        print(i,entrylens[foo],len(entrydates[foo]),foo,entryheads[foo],entrydates[foo])
        
        
        
        
        

newdates1 850 newdates 499 newdates2 58 sents 0
17 9793 0 heracles-bio-1 Heracles or HERCULES  []
103 3853 0 romulus-bio-1 Ro'mulus  []
131 3357 0 dionysus-bio-1 Diony'sus  []
144 3156 0 odysseus-bio-1 Odysseus  []
164 2947 0 cabeiri-bio-1 CABEIRI  []
203 2542 0 argonautae-bio-1 Argonautae  []
223 2376 0 appuleius-bio-10 Appuleius  []
228 2347 0 epictetus-bio-1 Epicte'tus  []
231 2341 0 avienus-rufus-festus-bio-1 Avie'nus, Rufus Festus  []
235 2314 0 hermes-bio-1 Hermes  []
301 1966 0 poseidon-bio-1 Poseidon  []
305 1941 0 aeneas-bio-1 Aene'as  []
313 1880 0 nepos-cornelius-bio-1 Corne'lius Nepos  []
318 1859 0 dictys-cretensis-bio-1 Dictys Cretensis  []
325 1838 0 agamemnon-bio-1 Agamemnon  []
328 1831 0 jason-bio-1 Jason  []
331 1821 0 aenesidemus-bio-2 Aeneside'mus  []
339 1791 0 musae-bio-1 Musae  []
341 1757 0 achilles-bio-1 Achilles  []
360 1684 0 diomedes-bio-1 Diome'des  []
361 1684 0 pelops-bio-1 Pelops  []
377 1630 0 gregoras-nicephorus-bio-1 Nice'phorus Gre'goras  []
382 161

In [13]:
print(len(entrylens))

19902


In [14]:
abbrevs = {}
f = open('bio-work.xml')

for l in f:
    if(not re.search('<p>',l)):
        continue
    l = re.sub('<bibl[^>]+>[<]+</bibl>',' ',l)
    l = re.sub('<title[^>]+>[<]+</title>',' ',l)
    while(re.search('([A-Z][a-z][a-z][a-z]\.)',l)):
        m = re.search('([A-Z][a-z][a-z][a-z]\.)',l)
        curabb = m[1]
        if(curabb in abbrevs):
            abbrevs[curabb] = abbrevs[curabb] + 1
        else:
            abbrevs[curabb] = 1
        l = re.sub('([A-Z][a-z][a-z][a-z]\.)','',l)
    

f.close()

for foo in sorted(abbrevs,key=abbrevs.get,reverse=True):
    print(abbrevs[foo],foo)

760 Paus.
609 Bibl.
480 Hist.
394 Plin.
360 Plut.
278 Diod.
226 Suid.
151 Rome.
137 Comp.
110 Lips.
110 Phot.
107 Suet.
96 Thuc.
93 Pind.
86 Verg.
85 Anab.
82 Brut.
82 Anal.
67 Rhod.
66 Lugd.
65 Fast.
65 Caes.
62 Serv.
62 Poet.
61 Dict.
58 Orat.
54 Just.
51 Hymn.
49 Nero.
48 Diog.
46 Lond.
44 Phil.
43 Asia.
43 Hell.
42 Clem.
38 Plat.
36 Gaul.
36 Alex.
35 Carm.
34 Anth.
34 Troy.
33 Oxon.
33 Epit.
32 Vell.
30 Chil.
30 Amst.
29 Frag.
29 Orph.
29 Flor.
28 Bell.
27 Mart.
27 Inst.
26 Soph.
26 Pyth.
26 Macr.
23 Corp.
23 Verr.
23 Etym.
22 Curt.
21 Sext.
20 Cass.
20 Deor.
19 Fabr.
19 Oros.
19 Tusc.
19 Virg.
18 Thes.
17 Stob.
17 Patr.
17 Hirt.
16 Varr.
16 Vitr.
15 Elis.
15 Gens.
15 Erot.
15 Zeus.
15 Vict.
15 Poll.
15 Mith.
14 Phys.
14 Acad.
14 Plot.
14 Meth.
14 Codd.
14 Hisp.
14 Ibid.
13 Anim.
13 Pont.
13 Lect.
13 Suas.
13 Sept.
13 Sall.
12 Ajax.
12 Morb.
12 Serm.
12 Stat.
12 Cael.
11 Pers.
11 Fest.
11 Anon.
11 Horn.
11 Gent.
10 Voss.
10 Rhet.
10 Vesp.
10 Auct.
10 Bapt.
10 Guil.
10 Scut.
10 Orig

In [15]:
titles = {}
f = open('bio-work.xml')

for l in f:
    if(not re.search('<p>',l)):
        continue
    l = re.sub('<bibl[^>]+>[<]+</bibl>',' ',l)
    l = re.sub('<title[^>]+>[<]+</title>',' ',l)
    while(re.search('<hi rend="ital">([A-Z][a-z]+\. [^<]+)',l)):
        m = re.search('<hi rend="ital">([A-Z][a-z]+\. [^<]+)',l)
        print(l)
        curabb = m[1]
        if(curabb in titles):
            titles[curabb] = titles[curabb] + 1
        else:
            titles[curabb] = 1
        l = re.sub('<hi rend="ital">([A-Z][a-z]+\. [^<]+)','',l)
    

f.close()

for foo in sorted(titles,key=titles.get,reverse=True):
    print(titles[foo],foo)

In [16]:
xrefs = {}
f = open('bio-work.xml')

xrefcnt = 0
for l in f:
    if(not re.search('<p>',l)):
        continue
    l = re.sub('<bibl[^>]+>[<]+</bibl>',' ',l)
    l = re.sub('<title[^>]+>[<]+</title>',' ',l)
    while(re.search('\[<hi rend="smallcaps">([^<]+)',l)):
        m = re.search('\[<hi rend="smallcaps">([^<]+)',l)
        #print(l)
        curabb = m[1]
        if(curabb in xrefs):
            xrefs[curabb] = xrefs[curabb] + 1
        else:
            xrefs[curabb] = 1
        l = re.sub('\[<hi rend="smallcaps">([^<]+)','',l)
    

f.close()

for foo in sorted(xrefs,key=xrefs.get,reverse=True):
    print(xrefs[foo],foo)

10 MAXIMUS.
9 GALLUS.
8 FLACCUS.
8 RUFUS
5 MACER.
5 CONSTANTINUS VII.
5 LEPIDA
5 MAXIMUS
5 FLACCUS
5 PRISCUS
4 FUSCUS.
4 ALBINUS
4 CLUENTIUS.
4 LABEO.
4 COBIDAS.
4 MAMMAS.
4 STUDITA.
4 LONGINUS
4 SEVERUS
4 SABINUS
3 MILO.
3 NATALIS.
3 CRASSUS.
3 LARGUS
3 VERRES.
3 CHRYSOLORAS.
3 CATO
3 VARRO
3 PARDUS.
3 TORQUATUS.
3 CALECAS.
3 DAMASCENUS
3 CIVILIS
3 DIOCLETIANUS
3 EUTHYMIUS
3 TURPILIANUS.
3 GALLUS
3 BASSUS
3 SERENUS
3 SECUNDUS
3 BAREA
3 SILANUS
3 GALLICANUS
3 QUADRATUS
3 PROCULUS
3 POLLIO
3 SATURNINUS
2 ARBORIUS.
2 DEXTER.
2 JUVENTINUS.
2 ANTISTATES.
2 POLEMO.
2 PTOLEMAEUS APION.
2 MUTILUS.
2 REGULUS.
2 FORTUNATIANUS.
2 ACCIUS
2 TUCCA.
2 AVIENUS.
2 NEMESIANUS.
2 CORDUS.
2 CRISPINILLA.
2 VARENUS.
2 VESPASIANUS.
2 CIVILIS.
2 TACITUS.
2 CODINUS.
2 SAXA
2 PEPAGOMENUS.
2 FULGENTIUS.
2 FRONTINUS.
2 CERAMEUS.
2 GEMISTUS.
2 PLAUTIANUS
2 PALAMAS.
2 GETA.
2 ANAGNOSTES.
2 CAMATERUS.
2 CAMENIATA.
2 CLIMACUS.
2 EUGENICUS
2 PHILOPONUS.
2 BRYENNIUS.
2 FIRMICUS.
2 VESTINUS.
2 VALENTINIANUS.
2 LEO VI.


In [17]:
centuries = {}
    
def cntcents(fname):
    f = open(fname)

    xrefcnt = 0
    for l in f:
        if(not re.search('<p>',l)):
            continue
        l = re.sub('<bibl[^>]+>[<]+</bibl>',' ',l)
        l = re.sub('<title[^>]+>[<]+</title>',' ',l)
        matchs = '([a-zA-Z0-9]+)\s+(century)\s+([AB]\.\s*[DC]\.)'
        while(re.search(matchs,l)):
            m = re.search(matchs,l)
        #print(l)
            curabb = m[1] + ' ' + m[3]
            curabb = re.sub('\.\s+','.',curabb)
            if(curabb in centuries):
                centuries[curabb] = centuries[curabb] + 1
            else:
                centuries[curabb] = 1
            l = re.sub(matchs,'',l)
    

    f.close()
cntcents('bio-work.xml')
cntcents('../002/smithgeo-work.xml')

for foo in sorted(centuries):
    if(re.search('B',foo)):
        print('centval["'+foo+'"] = "-100-101"')
    else:
        print('centval["'+foo+'"] = "101-100"')

centval["12th A.D."] = "101-100"
centval["4th A.D."] = "101-100"
centval["4th B.C."] = "-100-101"
centval["6th B.C."] = "-100-101"
centval["7th A.D."] = "101-100"
centval["7th B.C."] = "-100-101"
centval["9th A.D."] = "101-100"
centval["eighth B.C."] = "-100-101"
centval["eleventh B.C."] = "-100-101"
centval["fifth A.D."] = "101-100"
centval["fifth B.C."] = "-100-101"
centval["first A.D."] = "101-100"
centval["first B.C."] = "-100-101"
centval["fourth B.C."] = "-100-101"
centval["last B.C."] = "-100-101"
centval["ninth B.C."] = "-100-101"
centval["second B.C."] = "-100-101"
centval["seventh B.C."] = "-100-101"
centval["sixth A.D."] = "101-100"
centval["sixth B.C."] = "-100-101"
centval["tenth B.C."] = "-100-101"
centval["third A.D."] = "101-100"
centval["third B.C."] = "-100-101"
centval["twelfth B.C."] = "-100-101"


In [46]:
def getbestentry(query):
    query = re.sub('^([A-Za-z])\.\s+(.+)','\g<2> \g<1>',query)
    query = re.sub('[\,\.\s]+','-',query)
    queries = query.lower().split('-')
    if(len(queries)> 1):
        filts = '\\b' + queries[1] + '\\b'
    else:
        filts = ''
    if(not queries[0] in singnameindex):
        print(queries[0],'not known')
        return
    workdict = singnameindex[queries[0]]
    retlist = sorted(workdict,key=workdict.get,reverse=True)
    
    #print(retlist[1:5])
    goods = []
    bads = []
    for foo in sorted(workdict,key=workdict.get,reverse=True):
        if(filts and not re.search(filts,foo)):
            #continue
            #print('\tfailfilt',end=' ')
            bads.append(foo)
        else:
            goods.append(foo)
        #print(foo,workdict[foo])
    print('good',goods)
    print('failed',bads)

getbestentry('SCHOLASTICUS')

good ['cometas-scholasticus-bio-1', 'athanasius-scholasticus-bio-1', 'eratosthenes-scholasticus-bio-1', 'arabius-scholasticus-bio-1', 'athanasius-scholasticus-bio-2']
failed []


In [56]:
import re
f = open('bio-work.xml')

i = 0
j = 0
k = 0
firstpara = 0
for l in f:
    m = re.search('entry"\s+xml:id="([^"]+-)([0-9]+)"',l)
    if(m):
        curentryid = m[1]+m[2]
        firstpara = 1
        expectnum = m[2]
        basename = re.sub('-bio.*$','',curentryid)
    
    
    if(re.search('<head>',l)):
        workl = re.sub('\s+or\s+<pers.+','',l)
    else:
        workl = l
    m = re.search('(<head>|<label[^>]*>).*<persName[^>]*>(.+)</persName>',workl)
    if(m):
        curname = m[2].lower()
        curname = re.sub('</persname>.+','',curname)
        curname = re.sub('<[^>]+>','',curname)
        curname = re.sub("'",'',curname)
        curname = re.sub('[\s\.,]+','-',curname)
        curname = re.sub('-$','',curname)
        if(not curname == basename):
            k = k + 1
            print('mismatch',k,curentryid,curname)
    m = re.search('([A-Z][a-z]+\s+[Gg]ens)',l)
    if(firstpara and re.search('-1$',curentryid) and m):
        j = j + 1
        curgens = m[1]
        #print(j,'gens',curentryid,curgens,l)
        
        
    m = re.search('<p>([0-9]+)\.',l)
    if(1 and m and firstpara and not m[1] == expectnum):
        i = i + 1
        #print(i,curentryid,l)
        
    if(re.search('<p>',l)):
        firstpara = 0

f.close()

mismatch 1 abgarus-bio-1 agbarus
mismatch 2 aburia-gens-1 aburia-gens
mismatch 3 aburia-gens-bio-2 c-aburius
mismatch 4 aburia-gens-bio-3 m-aburius
mismatch 5 acherusia-geo-1 acherusia
mismatch 6 aeolides-bio-1 aeolus
mismatch 7 aeropus-bio-2 aeropus-i
mismatch 8 aeschylus-bio-2 aeschylus-of-alexandria
mismatch 9 aeschylus-bio-4 aeschylus-of-rhodes
mismatch 10 agapetus-bio-2 agapetus-st
mismatch 11 agrippa-herodes-i-bio-1 herodes-i-agrippa
mismatch 12 agrippa-herodes-ii-bio-1 herodes-ii-agrippa
mismatch 13 ahala-bio-8 ahenobarbus
mismatch 14 albinius-bio-1 l-albinius
mismatch 15 albinovanus-p-tullius-bio-1 albinus
mismatch 16 alcaeus-bio-5 alcaeus-of-messene
mismatch 17 alcaeus-bio-6 alcaeus-of-mytilene
mismatch 18 alexander-bio-10 st-alexander-of-alexandria
mismatch 19 alexander-bio-12 alexander-of-antiochia
mismatch 20 alexander-bio-22 alexander-st
mismatch 21 alexander-i-egypt-bio-1 alexander-i
mismatch 22 alexander-bio-30 alexander-lychnus
mismatch 23 alexander-the-great-bio-1 alex

mismatch 184 fabricius-bio-1 l-fabricius
mismatch 185 fadius-bio-2 c-fadius
mismatch 186 falconia-bio-proba-1 falconia-proba
mismatch 187 faustus-reiensis-bio-3 faustus
mismatch 188 firmianus-symposius-caelius-bio-1 firmianus-symposius-caelius&gt;
mismatch 189 firmicus-maternus-julius-bio-1 firmicus-maternus-julius&gt;
mismatch 190 flavus-triciptinus-lucretius-bio-1 flavus-triciptinus-lucretius&gt;
mismatch 191 fufius-bio-2 m-fufius
mismatch 192 gabinius-bio-1 a-?-gabinius
mismatch 193 gabinius-bio-3 a-?-gabinius
mismatch 194 galba-bio-2 p-sulpicius-galba
mismatch 195 gallus-c-aelius-bio-1 gallus-c-?-aelius
mismatch 196 geminius-bio-1 c-geminius
mismatch 197 georgius-bio-8 georgius-branas
mismatch 198 georgius-bio-9 georgius-bryennius
mismatch 199 georgius-bio-10 georgius-buraphus
mismatch 200 georgius-bio-11 georgius-chumnus
mismatch 201 georgius-bio-12 georgius-cocalas
mismatch 202 georgius-bio-13 georgius-drosus
mismatch 203 georgius-bio-14 georgius-euphorbenus
mismatch 204 georgius

mismatch 406 laeca-porcius-bio-1 laeca-p-porcius
mismatch 407 laeca-porcius-bio-2 laeca-m-porcius
mismatch 408 tlg-0918 lamynthius
mismatch 409 lari-scolus-accoleius-bio-1 lari-scolus-accoleius&gt;
mismatch 410 lecanius-bio-1 c-lecanius
mismatch 411 leo-i-flavius-bio-1 flavius&gt;-leo-i
mismatch 412 leo-iii-flavius-bio-1 leo-iii
mismatch 413 leo-iv-flavius-bio-1 leo-iv
mismatch 414 leo-v-flavius-armenus-bio-1 leo-v
mismatch 415 leo-vi-flavius-bio-1 flavius&gt;-leo-vi
mismatch 416 leo-bio-10 leo-aegyptius
mismatch 417 leo-bio-12 leo-asinus
mismatch 418 leo-bio-20 leo-diaconus
mismatch 419 leo-bio-22 leo-grammaticus
mismatch 420 leo-bio-24 leo-magentenus
mismatch 421 leo-bio-25 leo-maleinus
mismatch 422 leo-bio-29 leo-philosophus
mismatch 423 leo-bio-30 leo-peripateticus
mismatch 424 leo-bio-32 leo-pythagoricus
mismatch 425 leo-bio-33 leo-rhetor
mismatch 426 leo-bio-34 leo-sapiens
mismatch 427 leo-bio-35 leo-stypiota
mismatch 428 leontius-bio-17 leontius-mechanicus
mismatch 429 leontius-

mismatch 615 priscus-servilius-bio-7 priscus-q-servilius
mismatch 616 priscus-senecio-q-sosius-bio-1 priscus-senecio-q-sosius&gt;
mismatch 617 probus-bio-2 m-valerius-probus
mismatch 618 prorsa-bio-1 proserpina
mismatch 619 prosper-bio-1 prosper-aquitanus
mismatch 620 psellus-bio-3 michael-psellus
mismatch 621 psellus-bio-4 michael-psellus
mismatch 622 ptolemaeus-bio-25 ptolemaeus-philadelphus
mismatch 623 pupienus-maximus-m-clodius-bio-1 pupienus-maximus-m-clodius&gt;
mismatch 624 quintillus-plautius-bio-2 quintillus-m-plautius
mismatch 625 romanus-lecapenus-bio-1 romanus-i-lecapenus&gt;
mismatch 626 romanus-iii-argyrus-bio-1 romanus-iii-argyrus&gt;
mismatch 627 roscius-bio-1 sex-roscius
mismatch 628 rufus-sextilius-bio-1 rufus-p-sextilius
mismatch 629 rufus-sextilius-bio-2 rufus-c-sextilius
mismatch 630 rumi-lia-rumina-bio-1 rumi-lia-rumina&gt;
mismatch 631 rutilius-numatianus-claudius-bio-1 rutilius-numatianus-claudius&gt;
mismatch 632 sabi-nus-claudius-bio-1 sabi-nus-claudius&gt;
m